In [1]:
from tkinter import ttk, filedialog
from PIL import Image, ImageTk
from PIL import ImageOps
import tkinter as tk
import numpy as np
import cv2

### IMAGE PROCESSING LOGIC ###


# NOTE: Code below is for functions created in case cv2 wasn't allowed (output should result the same)
# def median_filter(image, kernel_size):
#     pad_size = kernel_size // 2
#     padded_image = cv2.copyMakeBorder(image, pad_size, pad_size, pad_size, pad_size, cv2.BORDER_REPLICATE)
#     result = np.zeros_like(image)

#     for i in range(image.shape[0]):
#         for j in range(image.shape[1]):
#             window = padded_image[i:i+kernel_size, j:j+kernel_size].flatten()
#             result[i, j] = np.median(window)

#     return result

# def rgb_to_grayscale(image):
#     if len(image.shape) == 3 and image.shape[2] == 3:
#         grayscale_image = 0.299 * image[:, :, 0] + 0.587 * image[:, :, 1] + 0.114 * image[:, :, 2]

#         # NOTE: convert to 8 bit
#         grayscale_image = grayscale_image.astype(np.uint8)

#         return grayscale_image
#     else:
#         raise ValueError("Input image should be in RGB format (3 channels).")

# def kmeans(X, k, max_iters=100, tol=1e-4):
#     centroids = X[np.random.choice(len(X), k, replace=False)]

#     for _ in range(max_iters):
#         labels = np.argmin(np.linalg.norm(X[:, np.newaxis] - centroids, axis=2), axis=1)
#         new_centroids = np.array([X[labels == i].mean(axis=0) for i in range(k)])
        
#         if np.linalg.norm(new_centroids - centroids) < tol:
#             break

#         centroids = new_centroids
#     return labels, centroids

# def oil_painting_effect(image_path, brush_size=10, intensity=5, quantization_levels=16):
#     original_image = cv2.imread(image_path)

#     if brush_size % 2 == 0:
#         brush_size += 1

#     # median filter to reduce noise
#     median_filtered = median_filter(original_image, brush_size)

#     # convert the image to grayscale
#     gray_image = custom_rgb_to_grayscale(median_filtered)

#     # apply color quantization
#     quantized_image = cv2.cvtColor(median_filtered, cv2.COLOR_BGR2RGB)
#     Z = quantized_image.reshape((-1, 3))

#     # convert to np.float32
#     Z = np.float32(Z)

#     # apply k-means
#     labels, centers = kmeans(Z, quantization_levels)

#     # reconstruct image
#     centers = np.uint8(centers)
#     quantized_image = centers[labels.flatten()]
#     quantized_image = quantized_image.reshape(original_image.shape)

#     # apply intensity to quantized image
#     intensity_matrix = np.ones(original_image.shape, dtype="uint8") * intensity
#     oil_painting = cv2.add(quantized_image, intensity_matrix)
    
#     return oil_painting

def oil_painting_effect(image_path, brush_size, intensity, quantization_levels):
    original_image = cv2.imread(image_path)
    
    if brush_size % 2 == 0:
        brush_size += 1

    median_filtered = cv2.medianBlur(original_image, brush_size)
    gray_image = cv2.cvtColor(median_filtered, cv2.COLOR_BGR2GRAY)
    quantized_image = cv2.cvtColor(median_filtered, cv2.COLOR_BGR2RGB)
    
    Z = quantized_image.reshape((-1, 3))
    Z = np.float32(Z)
    
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.2)
    K = quantization_levels
    _, labels, centers = cv2.kmeans(Z, K, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
    centers = np.uint8(centers)
    
    quantized_image = centers[labels.flatten()]
    quantized_image = quantized_image.reshape(original_image.shape)
    
    intensity_matrix = np.ones(original_image.shape, dtype="uint8") * intensity
    oil_painting = cv2.add(quantized_image, intensity_matrix)    
    
    return oil_painting

### UI ###
def get_file_path():
    file_path = filedialog.askopenfilename(filetypes=[])
    return file_path

def upload_image():
    file_path = get_file_path()

    if file_path:
        original_image = Image.open(file_path)
        original_image = original_image.resize((500, 500))
        original_photo = ImageTk.PhotoImage(original_image)

        original_image_label.config(image=original_photo)
        original_image_label.image = original_photo
        original_image_label.file_path = file_path
        original_image_label.place(x=50, y=50)

        altered_image_label.config(image=None)
        altered_image_label.image = None

def clear_image():
    original_image_label.config(image=None)
    original_image_label.image = None
    original_image_label.file_path = None

    altered_image_label.config(image=None)
    altered_image_label.image = None

def apply_oil_painting():
    if original_image_label.file_path:
        # NOTE: print statments to ensure code is not crashing...
        #       comment out print statments when not testing...
        print("Starting oil painting process...")

        file_path = original_image_label.file_path
        brush_size = brush_size_var.get()
        intensity = intensity_var.get()
        quantization_levels = quantization_levels_var.get()

        print("Reading the image...")
        oil_painting_result = oil_painting_effect(file_path, brush_size, intensity, quantization_levels)

        print("Converting the result to ImageTk format...")
        oil_painting_image = Image.fromarray(oil_painting_result)

        print("Resizing the image...")
        oil_painting_image = oil_painting_image.resize((500, 500))

        print("Creating ImageTk.PhotoImage...")
        oil_painting_photo = ImageTk.PhotoImage(oil_painting_image)

        print("Updating the altered image label...")
        altered_image_label.config(image=oil_painting_photo)
        altered_image_label.image = oil_painting_photo
        altered_image_label.place(x=750, y=50)  

        print("Oil painting process complete.")

if __name__ == "__main__":
    root = tk.Tk()
    root.title("Oil Converter")
    
    # Bad practice but easy to scale
    window_width = 1300
    window_height = 600
    root.geometry(f"{window_width}x{window_height}")
    root.resizable(False, False)

    original_image_label = tk.Label(root)
    original_image_label.pack(pady=10)

    altered_image_label = tk.Label(root)
    altered_image_label.pack(pady=10)

    button_width = 15 
    upload_button = tk.Button(root, text="Upload Image", command=upload_image, width=button_width)
    upload_button.pack(pady=10)

    clear_button = tk.Button(root, text="Clear Image", command=clear_image, width=button_width)
    clear_button.pack(pady=10)

    brush_size_var = tk.IntVar()
    brush_size_slider = tk.Scale(root, from_=0, to=100, orient="horizontal", variable=brush_size_var, label="Brush Size")
    brush_size_slider.pack(pady=10)

    intensity_var = tk.IntVar()
    intensity_slider = tk.Scale(root, from_=0, to=100, orient="horizontal", variable=intensity_var, label="Intensity")
    intensity_slider.pack(pady=10)

    quantization_levels_var = tk.IntVar()
    quantization_levels_slider = tk.Scale(root, from_=0, to=100, orient="horizontal", variable=quantization_levels_var, label="Quant Levels")
    quantization_levels_slider.pack(pady=10)

    apply_button = tk.Button(root, text="Apply Oil Painting", command=apply_oil_painting, width=button_width)
    apply_button.pack(pady=10)

    # UI cosmetics 
    upload_button.place(x=565, y=50)
    clear_button.place(x=565, y=90)

    brush_size_slider.place(x=600, y=140)  
    intensity_slider.place(x=600, y=215)  
    quantization_levels_slider.place(x=600, y=290)  
    
    apply_button.place(x=565, y=365)  

    root.mainloop()

Starting oil painting process...
Reading the image...
Converting the result to ImageTk format...
Resizing the image...
Creating ImageTk.PhotoImage...
Updating the altered image label...
Oil painting process complete.
